In [2]:
import torch

In [3]:
# get name dataset
with open('names.txt', 'r') as file:
    lines = [line.strip() for line in file]
# get unique chars
chars = sorted(list(set(''.join(lines))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [4]:
sz = len(stoi.items())
N = torch.zeros([sz, sz], dtype=torch.int32)
print(N.shape)
N.dtype

torch.Size([27, 27])


torch.int32

In [5]:
# create bigram matrix
for line in lines:
    chars = ['.'] + list(line) + ['.']
    for pr, sc in zip(chars, chars[1:]):
        idx1 = stoi[pr]
        idx2 = stoi[sc]
        N[idx1][idx2] += 1
N

tensor([[   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
         1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
          134,  535,  929],
        [6640,  556,  541,  470, 1042,  692,  134,  168, 2332, 1650,  175,  568,
         2528, 1634, 5438,   63,   82,   60, 3264, 1118,  687,  381,  834,  161,
          182, 2050,  435],
        [ 114,  321,   38,    1,   65,  655,    0,    0,   41,  217,    1,    0,
          103,    0,    4,  105,    0,    0,  842,    8,    2,   45,    0,    0,
            0,   83,    0],
        [  97,  815,    0,   42,    1,  551,    0,    2,  664,  271,    3,  316,
          116,    0,    0,  380,    1,   11,   76,    5,   35,   35,    0,    0,
            3,  104,    4],
        [ 516, 1303,    1,    3,  149, 1283,    5,   25,  118,  674,    9,    3,
           60,   30,   31,  378,    0,    1,  424,   29,    4,   92,   17,   23,
            0,  317,    1],
        [3983,  679,  121,  153,  384, 1271,   82,

In [20]:
# demonstration of broadcasting semantics and associated nuances
# we need the whole row to have the sum of the row for the row to be converted to a distribution
# so its important how we broadcast the tensor
dist = N.float()
sumarr = dist.sum(1, keepdim=True)
bad_sumarr = dist.sum(1)
# print(sumarr.shape)
# print(bad_sumarr.shape)
dummy_arr = torch.zeros([sz, sz])
correct_arr = dummy_arr + sumarr
# print(correct_arr)
bad_dist = dummy_arr + bad_sumarr
# print(bad_dist)

#actual useful code
dist = dist / sumarr

In [21]:
# sample using multinomial
gen = torch.Generator().manual_seed(2147483647)

for i in range(20):
    outvec= []
    idx = 0
    while True:
        d = dist[idx]
        idx = torch.multinomial(d, num_samples=1, replacement=True, generator=gen).item()
        outvec.append(itos[idx])
        if idx == 0:
            break
    print(''.join(outvec))

junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.
dedainrwieta.
ssonielylarte.
faveumerifontume.
phynslenaruani.
core.
yaenon.
ka.
jabdinerimikimaynin.
anaasn.
ssorionsush.
